# Final Pipeline
## Author: Robert Forristall

This document implements the final complete implementation of my work for FIU Spring 2024 CEN 5082 Project: Using Neuromancer tool to integrate physical constraints for an HPC Application (Power Grid)

In [299]:
# Case Generation
import os
import re
import random as rand
import math
import shutil
from tqdm import tqdm

# Dataset Preparation
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import string
import numpy as np
from neuromancer.dataset import DictDataset

# Model Generation
from neuromancer.system import Node
from neuromancer.loss import PenaltyLoss
from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.constraint import variable

## Case Generation

This section of code declares all of the functions that are used for the generation of new cases that can be used to train a neural network model

In [300]:
def sampleLowBound(pd, t):
    return (1-t)*pd;

def sampleHighBound(pd, t):
    return (1+t)*pd;

def sampleLoadValue(pd, t):
    if (pd != 0):
        low = sampleLowBound(pd, t)*100
        high = sampleHighBound(pd, t)*100
        if (low < high):
            return rand.randrange(math.floor(low), math.ceil(high))/100
        else:
            return rand.randrange(math.floor(high), math.ceil(low))/100
    else:
        return 0

def get_function_name(fileStr, index):
    return re.findall("function .* = .*", fileStr)[0] + f"_{index}"

def get_mpc_version(fileStr):
    return re.findall("mpc.version = '\d';", fileStr)[0]

def get_mpc_base(fileStr):
    return re.findall("mpc.baseMVA = \d*;", fileStr)[0]

def get_bus_data(fileStr, delta):
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    return_str = ""
    return_str += bus_data[0] + "\n"
    for i in range(1, len(bus_data)-1):
        current_row = bus_data[i].split("\t")
        current_row[3] = str(sampleLoadValue(float(current_row[3]), delta))
        current_row[4] = str(sampleLoadValue(float(current_row[4]), delta))
        return_str += "\t".join(current_row) + "\n"
    return_str += bus_data[-1] + "\n"
    return return_str

def get_gen_data(fileStr):
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    return_str = ""
    return_str += gen_data[0] + "\n"
    for i in range(1, len(gen_data)-1):
        return_str += gen_data[i] + "\n"
    return_str += gen_data[-1] + "\n"
    return return_str

def get_branch_data(fileStr):
    pattern = r"mpc.branch = \[[\n|\d|\t|;|.|-]*\];"
    branch_data = re.findall(pattern, fileStr)[0].split("\n")
    return_str = ""
    return_str += branch_data[0] + "\n"
    for i in range(1, len(branch_data)-1):
        return_str += branch_data[i] + "\n"
    return_str += branch_data[-1] + "\n"
    return return_str

def get_gencost_data(fileStr):
    pattern = r"mpc.gencost = \[[\n|\d|\t|;|.|-]*\];"
    gencost_data = re.findall(pattern, fileStr)[0].split("\n")
    return_str = ""
    return_str += gencost_data[0] + "\n"
    for i in range(1, len(gencost_data)-1):
        return_str += gencost_data[i] + "\n"
    return_str += gencost_data[-1] + "\n"
    return return_str

def get_full_case(fileStr, index, delta):
    case_str = "\n\n".join([
        get_function_name(fileStr, index),
        get_mpc_version(fileStr),
        get_mpc_base(fileStr),
        get_bus_data(fileStr, delta),
        get_gen_data(fileStr),
        get_branch_data(fileStr),
        get_gencost_data(fileStr)
    ])
    return case_str

def get_case_as_string(case_file):
    with open(case_file, "r") as file:
        return file.read()

def create_test_cases(fileStr, total_number_of_cases, delta):
    if not os.path.exists("generated_cases"):
        print("Generating New Cases...")
        os.mkdir("generated_cases")
        for i in tqdm(range(total_number_of_cases)):
            with open(f"generated_cases/GeneratedCase{i}.m", "w") as file:
                file.write(get_full_case(fileStr, i, delta))
    else:
        print("Cases already generated, skipping...")
        # shutil.rmtree("generated_cases")
        # os.mkdir("generated_cases")

## Run generated cases through MIPS

### In future iterations this stage will be automated by implementing MIPS in python

Run the generated files through MIPS and save the following for each case in its own excel within its own directory inside 'mips_results/' + the variable name lowercase:
- Optimization vector X (Va, Vm, Pg, Qg), saved in 'mips_results/x'
- Langrangian equality metric Lambda, saved in 'mips_results/lambda'
- Slack variable vector Z, saved in 'mips_results/z'
- Langrangian inequality metric Mu, saved in 'mips_results/mu'

## Prepare Train/Test Data (MTL Solo)

In [301]:
def get_bus_count(fileStr):
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    return len(bus_data)-2

def get_gen_count(fileStr):
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    return len(gen_data)-2

def get_pds_from_case(file):
    fileStr = get_case_as_string(file)
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    pds = []
    for i in range(1, len(bus_data)-1):
        current_row = bus_data[i].split("\t")
        pds.append(current_row[3])
    return pds

def get_qds_from_case(file):
    fileStr = get_case_as_string(file)
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    qds = []
    for i in range(1, len(bus_data)-1):
        current_row = bus_data[i].split("\t")
        qds.append(current_row[4])
    return qds

def build_pds_csv(bus_size):
    print("Building pds...")
    pd_data = pd.DataFrame(columns = [f"bus_{x+1}" for x in range(bus_size)]);
    for file in tqdm(os.listdir("generated_cases")):
        pd_data.loc[len(pd_data.index)] = get_pds_from_case("generated_cases/"+file)
    pd_data.to_csv("bus_Pds.csv", index=False)

def build_qds_csv(bus_size):
    print("Building qds...")
    qd_data = pd.DataFrame(columns = [f"bus_{x+1}" for x in range(bus_size)]);
    for file in tqdm(os.listdir("generated_cases")):
        qd_data.loc[len(qd_data.index)] = get_qds_from_case("generated_cases/"+file)
    qd_data.to_csv("bus_Qds.csv", index=False)

def load_data_MTL(bus_dir, bus_size, gen_size):
    # Load Bus Input Data
    if not os.path.exists("bus_Pds.csv"):
        build_pds_csv(bus_size)
    if not os.path.exists("bus_Qds.csv"):
        build_qds_csv(bus_size)
    Pds = pd.read_csv(bus_dir + 'bus_Pds.csv').to_numpy()
    Qds = pd.read_csv(bus_dir + 'bus_Qds.csv').to_numpy()
    input_data = []
    
    # Load Bus Label Data
    Va = []
    Vm = []
    Pg = []
    Qg = []
    Lambda = []
    Z = []
    Mu = []
    for file in tqdm(os.listdir(bus_dir + 'mips_results/x')):
        case_id = int(file.strip(string.ascii_letters + "."))
        input_data.append(np.concatenate((Pds[case_id], Qds[case_id]), axis=0))
        x_data = pd.read_csv(bus_dir + 'mips_results/x/' + file, header=None).to_numpy().flatten()
        Va.append(x_data[0:bus_size])
        Vm.append(x_data[bus_size:bus_size*2])
        Pg.append(x_data[bus_size*2:(bus_size*2) + gen_size])
        Qg.append(x_data[(bus_size*2) + gen_size:(bus_size*2) + (gen_size*2)])
        # Lambda.append(json.load(open('mips_results/lambda/myCase' + str(case_id) + '.json'))["eqnonlin"])
        Lambda.append(pd.read_csv(bus_dir + 'mips_results/lambda/' + file, header=None).to_numpy().flatten())
        Z.append(pd.read_csv(bus_dir + 'mips_results/z/' + file, header=None).to_numpy().flatten())
        Mu.append(pd.read_csv(bus_dir + 'mips_results/mu/' + file, header=None).to_numpy().flatten())
    return (input_data, Va, Vm, Pg, Qg, Lambda, Z, Mu)

# Craate Dataset
class CustomOpfMultiTaskDataset(Dataset):

    def __init__(self, x_data, Va, Vm, Pg, Qg, Lambda, Z, Mu):
        self.x_data = np.array(x_data).astype(np.float32)
        self.Va = np.array(Va).astype(np.float32)
        self.Vm = np.array(Vm).astype(np.float32)
        self.Pg = np.array(Pg).astype(np.float32)
        self.Qg = np.array(Qg).astype(np.float32)
        self.Lambda = np.array(Lambda).astype(np.float32)
        self.Z = np.array(Z).astype(np.float32)
        self.Mu = np.array(Mu).astype(np.float32)

    def __len__(self):
        return self.x_data.shape[0]

    def __getitem__(self, index):
        return {
            'x_val': self.x_data[index],
            'Va': self.Va[index],
            'Vm': self.Vm[index],
            'Pg': self.Pg[index],
            'Qg': self.Qg[index],
            'Lambda': self.Lambda[index],
            'Z': self.Z[index],
            'Mu': self.Mu[index]
        }

def create_data_loaders(split_index, batch_size, data_tuple, bus_size):

    train_dataloader = DataLoader(
        CustomOpfMultiTaskDataset(
            data_tuple[0][0:split_index],
            data_tuple[1][0:split_index],
            data_tuple[2][0:split_index],
            data_tuple[3][0:split_index],
            data_tuple[4][0:split_index],
            data_tuple[5][0:split_index],
            data_tuple[6][0:split_index],
            data_tuple[7][0:split_index],
        ), shuffle=True, batch_size=batch_size
    )

    validate_dataloader = DataLoader(
        CustomOpfMultiTaskDataset(
            data_tuple[0][split_index:],
            data_tuple[1][split_index:],
            data_tuple[2][split_index:],
            data_tuple[3][split_index:],
            data_tuple[4][split_index:],
            data_tuple[5][split_index:],
            data_tuple[6][split_index:],
            data_tuple[7][split_index:],
        ), shuffle=False, batch_size=batch_size
    )

    return (train_dataloader, validate_dataloader)

def get_MTL_solo_data(bus_size, gen_size):
    if (not os.path.exists(f"trainData.pth") or not os.path.exists(f"validData.pth")):
        print("Data Loaders for MTL solo not found, loading from raw data\n")
        (train_dataloader, validate_dataloader) = create_data_loaders(8000, 32, load_data_MTL("", bus_size, gen_size), bus_size)
        torch.save(train_dataloader, "trainData.pth")
        torch.save(validate_dataloader, "validData.pth")
    else:
        print("Data Loaders for MTL solo found, loading from .pth files\n")
        train_dataloader = torch.load("trainData.pth")
        validate_dataloader = torch.load("validData.pth")

    return train_dataloader, validate_dataloader

## Prepare Train/Test Data (MTL With Neuromancer)

In [302]:
def convert_data_to_dict(Pds, Qds, Va, Vm, Pg, Qg, Lambda, Z, Mu, start_index, end_index):
        return {
        "Pd": torch.tensor(Pds[start_index:end_index], dtype=torch.float32),
        "Qd": torch.tensor(Qds[start_index:end_index], dtype=torch.float32),
        "Va": torch.tensor(Va[start_index:end_index], dtype=torch.float32),
        "Vm": torch.tensor(Vm[start_index:end_index], dtype=torch.float32),
        "Pg": torch.tensor(Pg[start_index:end_index], dtype=torch.float32),
        "Qg": torch.tensor(Qg[start_index:end_index], dtype=torch.float32),
        "Lambda": torch.tensor(Lambda[start_index:end_index], dtype=torch.float32),
        "Z": torch.tensor(Z[start_index:end_index], dtype=torch.float32),
        "Mu": torch.tensor(Mu[start_index:end_index], dtype=torch.float32)
    }

def load_data():
    train_data = torch.load("neuromancerTrainData.pth")
    dev_data = torch.load("neuromancerDevData.pth")
    test_data = torch.load("neuromancerTestData.pth")
    out_size = pd.read_csv("output_size.csv", header=None).to_numpy()[0][0]

    return train_data, dev_data, test_data, out_size
    

def create_neuromancer_data(bus_dir, bus_size, gen_size, train_percent, dev_percent, test_percent):
    # Load Bus Input Data
    if not os.path.exists("bus_Pds.csv"):
        build_pds_csv(bus_size)
    if not os.path.exists("bus_Qds.csv"):
        build_qds_csv(bus_size)
    Pds = pd.read_csv(bus_dir + 'bus_Pds.csv').to_numpy()
    Qds = pd.read_csv(bus_dir + 'bus_Qds.csv').to_numpy()
    input_data = []

    num_of_cases = Pds.shape[0]
    train_end_index = int(num_of_cases*train_percent)
    dev_end_index = int(train_end_index + (num_of_cases*dev_percent))
    test_end_index = num_of_cases
    print(train_end_index)
    print(dev_end_index)

    out_size = 0
    
    #Load Bus Label Data
    Va = []
    Vm = []
    Pg = []
    Qg = []
    Lambda = []
    Z = []
    Mu = []
    for file in tqdm(os.listdir(bus_dir + 'mips_results/x')):
        case_id = int(file.strip(string.ascii_letters + "."))
        # input_data.append(np.concatenate((Pds[case_id], Qds[case_id]), axis=0))
        x_data = pd.read_csv(bus_dir + 'mips_results/x/' + file, header=None).to_numpy().flatten()
        Va.append(x_data[0:bus_size])
        Vm.append(x_data[bus_size:bus_size*2])
        Pg.append(x_data[bus_size*2:(bus_size*2) + gen_size])
        Qg.append(x_data[(bus_size*2) + gen_size:(bus_size*2) + (gen_size*2)])
        # Lambda.append(json.load(open('mips_results/lambda/myCase' + str(case_id) + '.json'))["eqnonlin"])
        lambda_data = pd.read_csv(bus_dir + 'mips_results/lambda/' + file, header=None).to_numpy().flatten()
        Lambda.append(lambda_data)
        z_data = pd.read_csv(bus_dir + 'mips_results/z/' + file, header=None).to_numpy().flatten()
        Z.append(z_data)
        mu_data = pd.read_csv(bus_dir + 'mips_results/mu/' + file, header=None).to_numpy().flatten()
        Mu.append(mu_data)
        if out_size == 0:
            out_size = x_data.shape[0] + lambda_data.shape[0] + z_data.shape[0] + mu_data.shape[0]
    # return (input_data, Va, Vm, Pg, Qg, Lambda, Z, Mu)

    train_dict = convert_data_to_dict(Pds, Qds, Va, Vm, Pg, Qg, Lambda, Z, Mu, 0, train_end_index)
    dev_dict = convert_data_to_dict(Pds, Qds, Va, Vm, Pg, Qg, Lambda, Z, Mu, train_end_index, dev_end_index)
    test_dict = convert_data_to_dict(Pds, Qds, Va, Vm, Pg, Qg, Lambda, Z, Mu, dev_end_index, test_end_index)
    
    train_dict = DictDataset(train_dict, name='train')
    dev_dict = DictDataset(dev_dict, name='dev')
    test_dict = DictDataset(test_dict, name='test')
    
    train_loader = torch.utils.data.DataLoader(train_dict, batch_size=32, num_workers=0,
                                               collate_fn=train_dict.collate_fn, shuffle=True)
    dev_loader = torch.utils.data.DataLoader(dev_dict, batch_size=32, num_workers=0,
                                             collate_fn=dev_dict.collate_fn, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dict, batch_size=32, num_workers=0,
                                             collate_fn=test_dict.collate_fn, shuffle=True)
    
    return train_loader, dev_loader, test_loader, out_size

def get_neuromancer_data(bus_size, gen_size):
    bus_dir = f"generated_cases"
    train_loader = None
    dev_loader = None
    test_loader = None
    if (not os.path.exists("neuromancerTrainData.pth")):
        print("Data Loaders for MTL in Neuromancer not found, loading from raw data\n")
        train_data, dev_data, test_data, out_size = create_neuromancer_data(f"", bus_size, gen_size, 0.7, 0.15, 0.15)
        torch.save(train_data, "neuromancerTrainData.pth")
        torch.save(dev_data, "neuromancerDevData.pth")
        torch.save(test_data, "neuromancerTestData.pth")
        df = pd.DataFrame(columns=['output_size'])
        df.loc[len(df.index)] = [out_size]
        df.to_csv("output_size.csv", header=False, index=False)
    else:
        print("Data Loaders for MTL in Neuromancer found, loading from .pth files\n")
        train_data, dev_data, test_data, out_size = load_data()

    return train_data, dev_data, test_data, out_size

## Model Generation Helpers

In [303]:
def get_z_mu_size():
    return pd.read_csv("mips_results/z/"+os.listdir("mips_results/z")[0], header=None).to_numpy().shape[0]

def generate_layers(bus_size, gen_size, z_mu_size):
        # print(f"\nBus Size: {bus_size}, Gen Size: {gen_size}")
        layer_1_sizes = [x for x in range(bus_size*2, (bus_size*2) + ((int(bus_size/4))*5), int(bus_size/4))]
        layer_x_sizes_1 = [x for x in range((bus_size*2) + ((int(bus_size/4))*5), bus_size, -int(bus_size/4))]
        layer_x_sizes_1[-1] = bus_size
        layer_x_sizes_1[0] = layer_1_sizes[-1]
        layer_x_sizes_2 = [x for x in range((bus_size*2) + ((int(bus_size/4))*5), gen_size, -int(bus_size/4))]
        layer_x_sizes_2[-1] = gen_size
        layer_x_sizes_2[0] = layer_1_sizes[-1]
        layer_l_sizes = [x for x in range((bus_size*2) + ((int(bus_size/4))*5), (bus_size*2)+1, -int(bus_size/4))]
        layer_l_sizes[-1] = (bus_size*2)+1
        layer_l_sizes[0] = layer_1_sizes[-1]
        layer_z_sizes = [x for x in range((bus_size*2)+(gen_size*2), ((bus_size*2)+(gen_size*2))*4, int(bus_size/2))]+ [x for x in range(((bus_size*2)+(gen_size*2))*4, z_mu_size, -int(bus_size/2))]
        layer_z_sizes[-1] = z_mu_size
        layer_mu_sizes = [x for x in range(z_mu_size, ((bus_size*2)+(gen_size*2))*4, int(bus_size/2))]+ [x for x in range(((bus_size*2)+(gen_size*2))*4, z_mu_size, -int(bus_size/2))]
        layer_mu_sizes[-1] = z_mu_size
        return layer_1_sizes, layer_x_sizes_1, layer_x_sizes_2, layer_l_sizes, layer_z_sizes, layer_mu_sizes

def get_device():
    # print(torch.__version__)
    # print(torch.version.cuda)
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    # print(f"Using {device} device")
    # if device == "cuda":
        # print(torch.cuda.get_device_name(0))
    return device

## Model Generation

In [304]:
class TestMultiTaskNetSolo(torch.nn.Module):
    def __init__(self, bus_size, gen_size, z_mu_size):
        super(TestMultiTaskNetSolo, self).__init__()
        # Sahred layers (Input: 28, Output: 58)

        layer_1_sizes, layer_x_sizes_1, layer_x_sizes_2, layer_l_sizes, layer_z_sizes, layer_mu_sizes = generate_layers(bus_size, gen_size, z_mu_size)

        # print(layer_1_sizes)
        # print(layer_x_sizes_1)
        # print(layer_x_sizes_2)
        # print(layer_l_sizes)
        # print(layer_z_sizes)
        # print(layer_mu_sizes)
        
        shared_modules = []
        x_bus_modules_1 = []
        x_bus_modules_2 = []
        x_gen_modules_1 = []
        x_gen_modules_2 = []
        l_modules = []
        z_modules = []
        mu_modules = []

        for i in range(1, len(layer_1_sizes)):
            if i != len(layer_1_sizes)-1:
                shared_modules.append(torch.nn.Linear(layer_1_sizes[i-1], layer_1_sizes[i]))
                # shared_modules.append(torch.nn.ReLU())
            else:
                shared_modules.append(torch.nn.Linear(layer_1_sizes[i-1], layer_1_sizes[i]))

        for i in range(1, len(layer_x_sizes_1)):
            if i != len(layer_1_sizes)-1:
                x_bus_modules_1.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))
                # x_bus_modules_1.append(torch.nn.ReLU())
            else:
                x_bus_modules_1.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))

        for i in range(1, len(layer_x_sizes_1)):
            if i != len(layer_1_sizes)-1:
                x_bus_modules_2.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))
                # x_bus_modules_2.append(torch.nn.ReLU())
            else:
                x_bus_modules_2.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))

        for i in range(1, len(layer_x_sizes_2)):
            if i != len(layer_1_sizes)-1:
                x_gen_modules_1.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))
                # x_gen_modules_1.append(torch.nn.ReLU())
            else:
                x_gen_modules_1.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))

        for i in range(1, len(layer_x_sizes_2)):
            if i != len(layer_1_sizes)-1:
                x_gen_modules_2.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))
                # x_gen_modules_2.append(torch.nn.ReLU())
            else:
                x_gen_modules_2.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))

        for i in range(1, len(layer_l_sizes)):
            if i != len(layer_1_sizes)-1:
                l_modules.append(torch.nn.Linear(layer_l_sizes[i-1], layer_l_sizes[i]))
                # l_modules.append(torch.nn.ReLU())
            else:
                l_modules.append(torch.nn.Linear(layer_l_sizes[i-1], layer_l_sizes[i]))

        for i in range(1, len(layer_z_sizes)):
            if i != len(layer_1_sizes)-1:
                z_modules.append(torch.nn.Linear(layer_z_sizes[i-1], layer_z_sizes[i]))
                # z_modules.append(torch.nn.ReLU())
            else:
                z_modules.append(torch.nn.Linear(layer_z_sizes[i-1], layer_z_sizes[i]))

        for i in range(1, len(layer_mu_sizes)):
            if i != len(layer_1_sizes)-1:
                mu_modules.append(torch.nn.Linear(layer_mu_sizes[i-1], layer_mu_sizes[i]))
                # mu_modules.append(torch.nn.ReLU())
            else:
                mu_modules.append(torch.nn.Linear(layer_mu_sizes[i-1], layer_mu_sizes[i]))
        
        self.net = torch.nn.Sequential(*shared_modules)
        
        self.n_features = bus_size*2
        self.net.fc = torch.nn.Identity()

        self.x_l_heads = torch.nn.ModuleList([])
        self.z_heads = torch.nn.ModuleList([])
        self.mu_heads = torch.nn.ModuleList([])
        
        self.x_l_heads.append(torch.nn.Sequential(
            *x_bus_modules_1
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_bus_modules_2
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_gen_modules_1
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_gen_modules_2
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *l_modules
        ))
        self.z_heads.append(torch.nn.Sequential(
            *z_modules
        ))
        self.mu_heads.append(torch.nn.Sequential(
            *mu_modules
        ))
        
    def forward(self, x):
        Shared_head = self.net(x)
        output = []
        for head in self.x_l_heads:
            output.append(head(Shared_head))
        for head in self.z_heads:
            output.append(head(torch.cat((output[0], output[1], output[2], output[3]), dim=1)))
        for head in self.mu_heads:
            output.append(head(output[5]))
        return output

def create_model_MTL_solo(bus_size, gen_size, z_mu_size, shouldPrint, batch_size=32):
    print("Creating MTL model for solo use...")
    model = TestMultiTaskNetSolo(bus_size, gen_size, z_mu_size).to(get_device())
    if (shouldPrint):
        summary(model, (batch_size,bus_size*2))
    return model

def train_model_MTL_solo(model, train_dataloader, validate_dataloader):
    # model = create_model(bus_size, False)
    n_epochs = 10
    device = get_device()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    loss_per_epoch = []
    val_loss_per_epoch = []
    acc_per_epoch = []
    val_acc_per_epoch = []

    lossStruct = torch.nn.L1Loss()
    for epoch in range(n_epochs):
        print(f"MTL Solo | Starting epoch: {epoch+1}")
        model.train(True)
        total_training_loss = 0
        total_training_acc = 0
        best_mse = 0
        for i, data in enumerate(train_dataloader):
            inputs = data["x_val"].to(device=device)
            Va_label = data["Va"].to(device=device)
            Vm_label = data["Vm"].to(device=device)
            Pg_label = data["Pg"].to(device=device)
            Qg_label = data["Qg"].to(device=device)
            L_label = data["Lambda"].to(device=device)
            Z_label = data["Z"].to(device=device)
            Mu_label = data["Mu"].to(device=device)
            optimizer.zero_grad()
            [Va_output, Vm_output, Pg_output, Qg_output, L_output, Z_output, Mu_output] = model(inputs)
            epoch_loss = []
            x = torch.cat((Va_output, Vm_output, Pg_output, Qg_output, L_output, Z_output, Mu_output), 1)
            y = torch.cat((Va_label, Vm_label, Pg_label, Qg_label, L_label, Z_label, Mu_label), 1)
            # loss = lossStruct.forward(x, y)
            loss = lossStruct(x, y) / 32
            loss.backward()
            optimizer.step()
            total_training_loss += loss
            train_acc = torch.sum(x == y)
        #     mse = ((torch.pow((x - y), 2)).sum()) / 32
        # if best_mse == 0 or best_mse < mse:
        #     best_mse = mse
        print("Training Loss after epoch: " + str(total_training_loss.item()))
        loss_per_epoch.append(total_training_loss.item())
        # print("Training Acc after epoch: " + str(best_mse))
        # acc_per_epoch.append(best_mse)

        total_validation_loss = 0
        best_mse = 0
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(validate_dataloader):
                inputs = data["x_val"].to(device=device)
                Va_label = data["Va"].to(device=device)
                Vm_label = data["Vm"].to(device=device)
                Pg_label = data["Pg"].to(device=device)
                Qg_label = data["Qg"].to(device=device)
                L_label = data["Lambda"].to(device=device)
                Z_label = data["Z"].to(device=device)
                Mu_label = data["Mu"].to(device=device)
                [Va_output, Vm_output, Pg_output, Qg_output, L_output, Z_output, Mu_output] = model(inputs)
                x = torch.cat((Va_output, Vm_output, Pg_output, Qg_output, L_output, Z_output, Mu_output), 1)
                y = torch.cat((Va_label, Vm_label, Pg_label, Qg_label, L_label, Z_label, Mu_label), 1)
                loss = lossStruct(x, y) / 32
                total_validation_loss += loss
            #     mse = ((torch.pow((x - y), 2)).sum()) / 32
            # if best_mse == 0 or best_mse < mse:
            #     best_mse = mse
            print("Validation Loss after epoch: " + str(total_validation_loss.item()))
            val_loss_per_epoch.append(total_validation_loss.item())
            # print("Validation Acc after epoch: " + str(best_mse))
            # val_acc_per_epoch.append(best_mse)
                
    print("Done Training, Loss: ")
    print(total_training_loss)
    return (model, loss_per_epoch, val_loss_per_epoch)

## Neuromancer Generation

In [310]:
class TestMultiTaskNetNeuromancer(torch.nn.Module):
    def __init__(self, bus_size, gen_size, z_mu_size):
        super(TestMultiTaskNetNeuromancer, self).__init__()
        # Sahred layers (Input: 28, Output: 58)

        layer_1_sizes, layer_x_sizes_1, layer_x_sizes_2, layer_l_sizes, layer_z_sizes, layer_mu_sizes = generate_layers(bus_size, gen_size, z_mu_size)

        # print(layer_1_sizes)
        # print(layer_x_sizes_1)
        # print(layer_x_sizes_2)
        # print(layer_l_sizes)
        # print(layer_z_sizes)
        # print(layer_mu_sizes)

        self.device = get_device()
        
        shared_modules = []
        x_bus_modules_1 = []
        x_bus_modules_2 = []
        x_gen_modules_1 = []
        x_gen_modules_2 = []
        l_modules = []
        z_modules = []
        mu_modules = []

        for i in range(1, len(layer_1_sizes)):
            if i != len(layer_1_sizes)-1:
                shared_modules.append(torch.nn.Linear(layer_1_sizes[i-1], layer_1_sizes[i]))
                # shared_modules.append(torch.nn.ReLU())
            else:
                shared_modules.append(torch.nn.Linear(layer_1_sizes[i-1], layer_1_sizes[i]))

        for i in range(1, len(layer_x_sizes_1)):
            if i != len(layer_1_sizes)-1:
                x_bus_modules_1.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))
                # x_bus_modules_1.append(torch.nn.ReLU())
            else:
                x_bus_modules_1.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))

        for i in range(1, len(layer_x_sizes_1)):
            if i != len(layer_1_sizes)-1:
                x_bus_modules_2.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))
                # x_bus_modules_2.append(torch.nn.ReLU())
            else:
                x_bus_modules_2.append(torch.nn.Linear(layer_x_sizes_1[i-1], layer_x_sizes_1[i]))

        for i in range(1, len(layer_x_sizes_2)):
            if i != len(layer_1_sizes)-1:
                x_gen_modules_1.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))
                # x_gen_modules_1.append(torch.nn.ReLU())
            else:
                x_gen_modules_1.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))

        for i in range(1, len(layer_x_sizes_2)):
            if i != len(layer_1_sizes)-1:
                x_gen_modules_2.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))
                # x_gen_modules_2.append(torch.nn.ReLU())
            else:
                x_gen_modules_2.append(torch.nn.Linear(layer_x_sizes_2[i-1], layer_x_sizes_2[i]))

        for i in range(1, len(layer_l_sizes)):
            if i != len(layer_1_sizes)-1:
                l_modules.append(torch.nn.Linear(layer_l_sizes[i-1], layer_l_sizes[i]))
                # l_modules.append(torch.nn.ReLU())
            else:
                l_modules.append(torch.nn.Linear(layer_l_sizes[i-1], layer_l_sizes[i]))

        for i in range(1, len(layer_z_sizes)):
            if i != len(layer_1_sizes)-1:
                z_modules.append(torch.nn.Linear(layer_z_sizes[i-1], layer_z_sizes[i]))
                # z_modules.append(torch.nn.ReLU())
            else:
                z_modules.append(torch.nn.Linear(layer_z_sizes[i-1], layer_z_sizes[i]))

        for i in range(1, len(layer_mu_sizes)):
            if i != len(layer_1_sizes)-1:
                mu_modules.append(torch.nn.Linear(layer_mu_sizes[i-1], layer_mu_sizes[i]))
                # mu_modules.append(torch.nn.ReLU())
            else:
                mu_modules.append(torch.nn.Linear(layer_mu_sizes[i-1], layer_mu_sizes[i]))
        
        self.net = torch.nn.Sequential(*shared_modules)
        
        self.n_features = bus_size*2
        self.net.fc = torch.nn.Identity()

        self.x_l_heads = torch.nn.ModuleList([])
        self.z_heads = torch.nn.ModuleList([])
        self.mu_heads = torch.nn.ModuleList([])
        
        self.x_l_heads.append(torch.nn.Sequential(
            *x_bus_modules_1
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_bus_modules_2
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_gen_modules_1
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *x_gen_modules_2
        ))
        self.x_l_heads.append(torch.nn.Sequential(
            *l_modules
        ))
        self.z_heads.append(torch.nn.Sequential(
            *z_modules
        ))
        self.mu_heads.append(torch.nn.Sequential(
            *mu_modules
        ))
        
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1).to(self.device)
        Shared_head = self.net(x)
        output = []
        for head in self.x_l_heads:
            output.append(head(Shared_head))
        for head in self.z_heads:
            output.append(head(torch.cat((output[0], output[1], output[2], output[3]), dim=1)))
        for head in self.mu_heads:
            output.append(head(output[5]))
        # return output
        return [x.to("cpu") for x in output]

def create_model_MTL_neuromancer(bus_size, gen_size, z_mu_size, shouldPrint, batch_size=32):
    print("Creating MTL model for neuromancer...")
    model = TestMultiTaskNetNeuromancer(bus_size, gen_size, z_mu_size).to(get_device())
    if (shouldPrint):
        summary(model, (batch_size,bus_size*2))
    return model

def get_Vm_min():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    Vm_mins = []
    for i in range(1, len(bus_data)-1):
        current_row = bus_data[i].split("\t")
        Vm_mins.append(float(current_row[-1].replace(";", "")))
    return Vm_mins

def get_Vm_max():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.bus = \[[\n|\d|\t|;|.|-]*\];"
    bus_data = re.findall(pattern, fileStr)[0].split("\n")
    Vm_maxs = []
    for i in range(1, len(bus_data)-1):
        current_row = bus_data[i].split("\t")
        Vm_maxs.append(float(current_row[-2]))
    return Vm_maxs
    
def get_Qg_min():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    Qg_mins = []
    for i in range(1, len(gen_data)-1):
        current_row = gen_data[i].split("\t")
        Qg_mins.append(float(current_row[4]))
    return Qg_mins
    
def get_Qg_max():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    Qg_maxs = []
    for i in range(1, len(gen_data)-1):
        current_row = gen_data[i].split("\t")
        Qg_maxs.append(float(current_row[3]))
    return Qg_maxs

def get_Pg_min():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    Pg_mins = []
    for i in range(1, len(gen_data)-1):
        current_row = gen_data[i].split("\t")
        Pg_mins.append(float(current_row[9]))
    return Pg_mins

def get_Pg_max():
    fileStr = get_case_as_string("generated_cases/"+os.listdir("generated_cases")[0])
    pattern = r"mpc.gen = \[[\n|\d|\t|;|.|-]*\];"
    gen_data = re.findall(pattern, fileStr)[0].split("\n")
    Pg_maxs = []
    for i in range(1, len(gen_data)-1):
        current_row = gen_data[i].split("\t")
        Pg_maxs.append(float(current_row[8]))
    return Pg_maxs

def create_neuromancer_problem(func):
    sol_map = Node(func, ["Pd", "Qd"], ["x"], name='map')

    Pd = variable("Pd")
    Qd = variable("Qd")
    Va = variable("Va")
    Vm = variable("Vm")
    Pg = variable("Pg")
    Qg = variable("Qg")
    Lambda = variable("Lambda")
    Z = variable("Z")
    Mu = variable("Mu")
    Va_i = variable("x")[0]
    Vm_i = variable("x")[1]
    Pg_i = variable("x")[2]
    Qg_i = variable("x")[3]
    Lambda_i = variable("x")[4]
    Z_i = variable("x")[5]
    Mu_i = variable("x")[6]
    
    # Objective function
    f1 = torch.abs(Vm-Vm_i) + torch.abs(Va-Va_i)
    f2 = torch.abs(Pg-Pg_i) + torch.abs(Qg-Qg_i)
    f3 = torch.abs(Lambda - Lambda_i)
    f4 = torch.abs(Z - Z_i)
    f5 = torch.abs(Mu - Mu_i)
    obj1 = f1.minimize(weight=0.35, name='obj1')
    obj2 = f2.minimize(weight=0.35, name='obj2')
    obj3 = f3.minimize(weight=0.10, name='obj3')
    obj4 = f4.minimize(weight=0.10, name='obj4')
    obj5 = f5.minimize(weight=0.10, name='obj5')
    
    # Constraints
    Q_con = 100.  # constraint penalty weights
    
    # con_1 = Q_con*(V_min <= Vm_i <= V_max)
    # con_1.name = 'c1'
    
    # constraints = [con_1]
    constraints = []

    for index, (min, max) in enumerate(zip(get_Vm_min(), get_Vm_max())):
        con = Q_con*(min <= Vm_i[:, [index]] <= max)
        con.name= 'c1_'+str(index)
        constraints.append(con)
    
    for index, (min, max) in enumerate(zip(get_Qg_min(), get_Qg_max())):
        con = Q_con*(min <= Qg_i[:, [index]] <= max)
        con.name = 'c2_'+str(index)
        constraints.append(con)
    
    for index, (min, max) in enumerate(zip(get_Pg_min(), get_Pg_max())):
        con = Q_con*(min <= Pg_i[:, [index]] <= max)
        con.name = 'c3_'+str(index)
        constraints.append(con)
    
    
    # constrained optimization problem construction
    objectives = [obj1, obj2, obj3, obj4, obj5]
    components = [sol_map]
    
    # create penalty method loss function
    loss = PenaltyLoss(objectives, constraints)
    # construct constrained optimization problem
    return Problem(components, loss)

def prepare_trainer(problem, train_data, dev_data, test_data):
    lr = 0.0001      # step size for gradient descent
    epochs = 10    # number of training epochs
    warmup = 5    # number of epochs to wait before enacting early stopping policy
    patience = 5  # number of epochs with no improvement in eval metric to allow before early stopping

    optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
    
    # define trainer
    return Trainer(
        problem,
        train_data,
        dev_data,
        test_data,
        optimizer,
        epochs=epochs,
        patience=patience,
        warmup=warmup)

def train_neuromancer_model(trainer):
    best_model = trainer.train()
    best_outputs = trainer.test(best_model)
    return best_model, best_outputs

## Main Pipeline Implementation

In [313]:
def full_pipeline(case_file, 
                  total_number_of_cases=10000, 
                  delta=0.1, 
                  test_percent=0.2, 
                  activations=[None, None, None, None, None],
                  dropouts=[(False, 0),(False, 0),(False, 0),(False, 0),(False, 0)],
                  engine="matlab"):
    fileStr = get_case_as_string(case_file)
    bus = (get_bus_count(fileStr), get_gen_count(fileStr))
    create_test_cases(fileStr, total_number_of_cases, delta)
    # # label_cases(engine)
    if not os.path.exists("mips_results"):
        print("Label cases to continue")
        return
    bus_dir = f"generated_cases/"
    MTL_solo_train = None
    MTL_solo_val = None
    neuromancer_train_loader = None
    neuromancer_dev_loader = None
    neuromancer_test_loader = None
    output_size = None
    MTL_solo_train, MTL_solo_val = get_MTL_solo_data(bus[0], bus[1])
    neuromancer_train_loader, neuromancer_dev_loader, neuromancer_test_loader, output_size = get_neuromancer_data(bus[0], bus[1])

    MTL_solo_model = create_model_MTL_solo(bus[0], bus[1], get_z_mu_size(), False, 32)
    trained_MTL_solo_model, MTL_solo_loss_per_epoch, MTL_solo_val_loss_per_epoch = train_model_MTL_solo(MTL_solo_model, MTL_solo_train, MTL_solo_val)
    
    MTL_in_neuromancer_model = create_model_MTL_neuromancer(bus[0], bus[1], get_z_mu_size(), False, 32)
    neuromancer_problem = create_neuromancer_problem(MTL_in_neuromancer_model)
    neuromancer_trainer = prepare_trainer(neuromancer_problem, neuromancer_train_loader, neuromancer_dev_loader, neuromancer_test_loader)
    neuromancer_best_model, neuromancer_best_outputs = train_neuromancer_model(neuromancer_trainer)
    neuromancer_problem.load_state_dict(neuromancer_best_model)

In [314]:
full_pipeline("case14.m")

Cases already generated, skipping...
Data Loaders for MTL solo found, loading from .pth files

Data Loaders for MTL in Neuromancer found, loading from .pth files

Creating MTL model for solo use...
MTL Solo | Starting epoch: 1
Training Loss after epoch: 2725.273681640625
Validation Loss after epoch: 686.6591186523438
MTL Solo | Starting epoch: 2
Training Loss after epoch: 2724.23828125
Validation Loss after epoch: 686.18896484375
MTL Solo | Starting epoch: 3
Training Loss after epoch: 2720.246826171875
Validation Loss after epoch: 684.2876586914062
MTL Solo | Starting epoch: 4
Training Loss after epoch: 2699.376953125
Validation Loss after epoch: 676.1289672851562
MTL Solo | Starting epoch: 5
Training Loss after epoch: 2658.9609375
Validation Loss after epoch: 660.732421875
MTL Solo | Starting epoch: 6
Training Loss after epoch: 2554.10986328125
Validation Loss after epoch: 620.3870239257812
MTL Solo | Starting epoch: 7
Training Loss after epoch: 2281.24853515625
Validation Loss after 